# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Vacay_df = pd.read_csv("../output_data/cities.csv", )
Vacay_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,gazanjyk,74,TM,1584498120,80,48.00,39.24,55.52,6.20
1,inhambane,21,MZ,1584498022,73,73.49,-23.86,35.38,13.44
2,ribeira grande,50,PT,1584498120,89,58.17,38.52,-28.70,10.85
3,bengkulu,97,ID,1584497867,70,85.77,-3.80,102.27,2.10
4,punta arenas,75,CL,1584498121,66,53.60,-53.15,-70.92,24.16
...,...,...,...,...,...,...,...,...,...
567,teeli,76,RU,1584498543,91,28.96,51.03,90.23,2.01
568,oriximina,100,BR,1584498543,87,78.66,-1.77,-55.87,5.86
569,la ronge,75,CA,1584498307,49,3.20,55.10,-105.28,2.24
570,san policarpo,65,PH,1584498544,81,80.10,12.18,125.51,9.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = Vacay_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = Vacay_df[["Lat", "Lng"]]


In [4]:
fig = gmaps.figure(layout={'width': '800px',
            'height': '400px',
            'padding': '3px',
            'border': '1px solid black'}, zoom_level=1.5, center = (40,90))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 max_intensity=maxhumidity,
                                 point_radius=3, )
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
Narrow_df = Vacay_df.loc[(Vacay_df["Temp"] > 70) & (Vacay_df["Temp"] < 85) & (Vacay_df["Wind Speed"]<10)
                       &(Vacay_df["Cloudiness"] == 0), :]
Narrow_df = Narrow_df.dropna(how='any')
Narrow_df.reset_index(inplace=True)
del Narrow_df['index']
hotel_df = pd.DataFrame(Narrow_df)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,caravelas,0,BR,1584498148,81,80.76,-17.71,-39.25,8.25
1,chiang klang,0,TH,1584498151,83,73.40,19.29,100.86,7.38
2,east london,0,ZA,1584498154,73,70.61,-33.02,27.91,6.89
3,vieux-habitants,0,GP,1584498159,88,73.40,16.06,-61.77,6.93
4,puerto ayora,0,EC,1584498183,77,80.01,-0.74,-90.35,8.01
5,gopalganj,0,IN,1584498186,36,74.46,26.47,84.43,9.01
6,pokhara,0,NP,1584498221,41,71.47,28.23,83.98,0.83
7,binzhou,0,CN,1584498223,29,71.01,37.37,118.02,5.01
8,airai,0,TL,1584498001,50,80.08,-8.93,125.41,5.86
9,whitianga,0,NZ,1584498252,46,75.00,-36.83,175.70,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
Hotelname = []


In [7]:
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
   
    data = response.json()
    try:
        Hotelname.append(data['results'][0]['name'])
    except:
        Hotelname.append("")
hotel_df["Hotel Name"] = Hotelname
hotel_df = hotel_df.dropna(how='any')
hotel_df
        

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,caravelas,0,BR,1584498148,81,80.76,-17.71,-39.25,8.25,Pousada dos Navegantes
1,chiang klang,0,TH,1584498151,83,73.40,19.29,100.86,7.38,Sangthongresort
2,east london,0,ZA,1584498154,73,70.61,-33.02,27.91,6.89,Tu Casa
3,vieux-habitants,0,GP,1584498159,88,73.40,16.06,-61.77,6.93,Gîtes Dardanelle
4,puerto ayora,0,EC,1584498183,77,80.01,-0.74,-90.35,8.01,Finch Bay Galapagos Hotel
5,gopalganj,0,IN,1584498186,36,74.46,26.47,84.43,9.01,SPOT ON 62129 Hotel Shabnam SPOT
6,pokhara,0,NP,1584498221,41,71.47,28.23,83.98,0.83,Hotel View Point
7,binzhou,0,CN,1584498223,29,71.01,37.37,118.02,5.01,Hampton By Hilton Binzhou
8,airai,0,TL,1584498001,50,80.08,-8.93,125.41,5.86,Juvinal Martin
9,whitianga,0,NZ,1584498252,46,75.00,-36.83,175.70,4.00,Beachside Resort Whitianga


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
names = hotel_df["Hotel Name"]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='3px', width='800px'))